In [225]:
from dotenv import load_dotenv
import os
from openai import OpenAI
from IPython.display import Markdown, display
import gradio as gr
import ast

In [226]:
load_dotenv(override=True)
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:3]}")
else:
    print("OpenRouter API Key not set (and this is optional)")

openrouter_url = "https://openrouter.ai/api/v1"

openrouter = OpenAI(base_url=openrouter_url, api_key=openrouter_api_key)

OpenRouter API Key exists and begins sk-


In [227]:
MODEL = 'google/gemini-2.5-flash-lite'

In [ ]:
## Utilities

def call_llm(model, system_message, user_message):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ]
    response = openrouter.chat.completions.create(model=model, messages=messages)
    return response.choices[0].message.content


def call_llm_with_history(model, system_message, history, user_message):
    messages = [{"role": "system", "content": system_message}] + history + [
        {"role": "user", "content": user_message}
    ]
    response = openrouter.chat.completions.create(model=model, messages=messages)
    return response.choices[0].message.content


def safe_eval(text):
    return ast.literal_eval(text)

In [264]:
def cat_agent(message):
    system_message = '''You are a helpful assistant that catergorizes the given message into two catergories.
                        Category 1 : General Query
                        Category 2 : Drug Information
                        If the message contains any drug or medicine name then put it into Category 2 : Drug Information
                        Return only the category decision of the whole message.
    '''
    category = call_llm(MODEL, system_message, message)
    print(category)
    return category

def reply_normal_agent(message,history):
    system_message = 'You are a helpful assistant that can answer questions and help with tasks.'
    response = call_llm_with_history(MODEL, system_message,history, message)
    return response


def paracetamol_agent(message):
    print('paracetamol_agent is running')
    system_message = '''You are a helpful assistant that answers questions only related to paracetamol and nothing else.
                        If any other drug is mentioned then completly ignore its mention and do not talk about it.
                        Return results in Dict format where the keys are different medicine names
                        Return ONLY a Python dict.
                        Do NOT wrap the response in backticks.
                        Do NOT use JSON.
                        Do NOT add explanations.
                        Example : user message : what is paracetamol. what is insulin
                                  assistant message : {'paracetamol' : 'details about paracetamol',
                                                        'insulin' : 'details about insulin'}'''
    raw_paractamol_output = call_llm(MODEL, system_message, message)
    paracetamol = safe_eval(raw_paractamol_output)
    return paracetamol['paracetamol']


def insulin_agent(message):
    print('insulin_agent is running')
    system_message = '''You are a helpful assistant that answers questions only related to insulin and nothing else.
                        If any other drug is mentioned then completly ignore its mention and do not talk about it.
                        Return results in Dict format where the keys are different medicine names
                        Return ONLY a Python dict.
                        Do NOT wrap the response in backticks.
                        Do NOT use JSON.
                        Do NOT add explanations.
                        Example : user message : what is paracetamol. what is insulin
                                  assistant message : {'paracetamol' : 'details about paracetamol',
                                                        'insulin' : 'details about insulin'}'''
    raw_insulin_output = call_llm(MODEL, system_message, message)
    insulin = safe_eval(raw_insulin_output)
    return insulin['insulin']

def drug_id(message):
    system_message = '''You are a helpful assistant that picks out all the drug names in a query and gives a list of the drug names present in the query.
                        Return only python list
                        Fix the typos as well if you find any'''
    drugs_raw = call_llm(MODEL, system_message, message)
    drugs = safe_eval(drugs_raw)
    print(drugs)

    results = []

    for drug in drugs:
        if drug.lower() == 'paracetamol':
            results.append(paracetamol_agent(message))

        elif drug.lower() == 'insulin' :
            results.append(insulin_agent(message)) 
        
        else :
            results.append(f"Invalid drug {drug}. I only answer questions related to paracetamol and insulin")

    return results



In [265]:
def orch(message,history) :
    catergory = cat_agent(message)
    if 'General Query' in catergory:
        return reply_normal_agent(message,history)
    elif 'Drug Information' in catergory:
        return drug_id(message)
    else :
        return "Sorry, I could not categorize your question. Please try rephrasing it."

In [ ]:
gr.ChatInterface(fn=orch, type="messages").launch()